<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric[0],metric[1],metric[2]],',')
    return {'loss': -metric[0], 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[82.95657006612396, 'False', 0.04536342608086675, 0.0348956953642386, 0.121408771116375]                               
,                                                                                                                      
[93.50468566837422, 'False', 0.04442482751307143, 0.034256622516556604, 0.11817531930440403]                           
,                                                                                                                      
[56.02600888724633, 'False', 0.04620414374787472, 0.03566059602649038, 0.12236855478111894]                            
,                                                                                                                      
[5.323964550315936, 'True', 0.04126758950343522, 0.032041390728477157, 0.11028920604874207]                            
,                                                                                                                      
[94.21805967305588, 'True', 0.0450171576

[45.143353300485714, 'True', 0.045819978051663844, 0.03537913907284796, 0.12098523238984293]                           
,                                                                                                                      
[10.21170430542612, 'True', 0.04483549639346289, 0.03475827814569559, 0.1191649311445208]                              
,                                                                                                                      
[13.822730889326921, 'True', 0.04552056849923391, 0.03511589403973532, 0.12219548864056698]                            
,                                                                                                                      
[20.0644955834894, 'True', 0.046090430868346556, 0.03558278145695399, 0.12208875576332733]                             
,                                                                                                                      
[59.03915536713354, 'True', 0.0456245068

KeyboardInterrupt: 

In [4]:
array = [[82.91899068705636, 'False', 0.0444901114931397, 0.03419370860927167, 0.11893605246979752]       #117.57/57                      
,                                                                                                                      
[52.20202066646283, 'False', 0.04554415610459907, 0.03516390728476848, 0.12064565100955689]                #103.54/48            
,                                                                                                                      
[7.228034453654379, 'True', 0.042850859750016936, 0.03323675496688766, 0.11472208878727952]                            
,                                                                                                                      
[47.79023882438347, 'True', 0.04596201175895313, 0.035445364238410805, 0.12192027063021034]                            
,                                                                                                                      
[4.953022267001969, 'False', 0.03964559279276326, 0.03087913907284789, 0.10480921299858056]                            
,                                                                                                                      
[79.33046314444026, 'True', 0.04479502409808972, 0.03454470198675517, 0.11887531110233153]                             
,                                                                                                                      
[45.78555195651614, 'False', 0.046480448468592624, 0.03586754966887447, 0.12431663584155332]                           
,                                                                                                                      
[78.68392633711339, 'False', 0.0450279054899607, 0.034723509933775085, 0.11989509878831335]                            
,                                                                                                                      
[86.96897748328931, 'False', 0.04479759543339232, 0.03451655629139104, 0.11968659981189164]                            
,                                                                                                                      
[49.8793904803881, 'True', 0.04642301609453759, 0.03576655629139095, 0.12380586595299334]                              
,                                                                                                                      
[78.887930366438, 'True', 0.044537378662282655, 0.03434105960264931, 0.11878194491644364]                              
,                                                                                                                      
[63.57817342466922, 'True', 0.04537882341488977, 0.03498178807947036, 0.1213184988778308]                              
,                                                                                                                      
[89.86656062368799, 'True', 0.04475548567411248, 0.03438907284768236, 0.12024893159557642]                             
,                                                                                                                      
[11.387746460292506, 'False', 0.04519243475468667, 0.035018211920530104, 0.1204812948884441]                           
,                                                                                                                      
[34.950364576984256, 'True', 0.04577214854401372, 0.035274834437086344, 0.1212239714163671]                            
,                                                                                                                      
[78.11491300750264, 'False', 0.045209531906551685, 0.034763245033112816, 0.12088019236018507]                          
,                                                                                                                      
[71.53722195641515, 'False', 0.045461944492338056, 0.03500496688741748, 0.12064802582876161]                           
,                                                                                                                      
[9.472775762671837, 'True', 0.044242972498592235, 0.034375827814569763, 0.11757621471762861]                           
,                                                                                                                      
[91.69032698860048, 'False', 0.044706699137507314, 0.03428476821192078, 0.12133587239827748]                           
,                                                                                                                      
[59.5129907584054, 'False', 0.045939495118349105, 0.03535761589404018, 0.12203045330601404]                            
,                                                                                                                      
[29.188149074750523, 'True', 0.04671624966009149, 0.03608774834437111, 0.12356432803912087]                            
,                                                                                                                      
[25.973882918960605, 'True', 0.04710960656657937, 0.03644701986754991, 0.12387344148331272]                            
,                                                                                                                      
[21.97609799818271, 'True', 0.046700275939598225, 0.03600827814569573, 0.12428715391381906]                            
,                                                                                                                      
[30.36801887717365, 'True', 0.046277222768743145, 0.03571026490066258, 0.12210645851188373]                            
,                                                                                                                      
[22.266658950957297, 'True', 0.046174990940863664, 0.03571688741721878, 0.12187289032178106]                           
,                                                                                                                      
[33.75327407988199, 'True', 0.04627762819784001, 0.035753311258278395, 0.12176555535447792]                            
,                                                                                                                      
[17.739635425247904, 'True', 0.04561297168987199, 0.03533112582781473, 0.12058190833028407]                            
,                                                                                                                      
[40.97752459084224, 'True', 0.04581829974298771, 0.03548178807947051, 0.12080833958547257]                             
,                                                                                                                      
[24.03783031552789, 'True', 0.04637396279456474, 0.0358907284768215, 0.12222186001402463]                              
,                                                                                                                      
[15.155744007741797, 'True', 0.04575142333289166, 0.0354155629139077, 0.12163547057453197]                             
,                                                                                                                      
[30.152386918567363, 'True', 0.046382934486393315, 0.035874172185430736, 0.12229398578915456]                          
,                                                                                                                      
[1.9208816348190219, 'True', 0.029734767719712098, 0.023296357615894485, 0.07925709509604197]                          
,                                                                                                                      
[39.23933752629402, 'True', 0.04621011880090312, 0.0356423841059605, 0.12348205718501704]                              
,                                                                                                                      
[61.32176734035707, 'True', 0.045170805904748215, 0.034912251655629396, 0.12014155901455371]                           
,                                                                                                                      
[53.585419643318616, 'True', 0.04582092863648335, 0.03536258278145736, 0.1210727369750712]                             
,                                                                                                                      
[27.20126043782994, 'True', 0.04645643473911397, 0.035960264900662564, 0.12338123782092351]                            
,                                                                                                                      
[43.85068143611936, 'True', 0.04614793523691199, 0.035577814569536785, 0.12296486759929298]                            
,                                                                                                                      
[3.4604517539829445, 'True', 0.03797994289297683, 0.029658940397351292, 0.0996258021259253]                            
,                                                                                                                      
[54.47410580765036, 'True', 0.0462010181833551, 0.03565066225165587, 0.12388219449974941]                              
,                                                                                                                      
[17.100419440069842, 'True', 0.04661959913464526, 0.036021523178808225, 0.12439554320452331]                           
,                                                                                                                      
[37.9331860236985, 'False', 0.04620141884996832, 0.03555132450331156, 0.12276857507921037]                             
,                                                                                                                      
[47.50082822222094, 'True', 0.04592659916434125, 0.03546026490066256, 0.12185332504236411]                             
,                                                                                                                      
[10.389564192501709, 'True', 0.044527172292405715, 0.03449006622516592, 0.11927144037262528]                           
,                                                                                                                      
[14.024145090367696, 'False', 0.045895274989290094, 0.03555463576158978, 0.1219737504372654]                           
,                                                                                                                      
[99.07950049723175, 'True', 0.044791501910448756, 0.03442715231788105, 0.12026628111137948]                            
,                                                                                                                      
[69.09097488511287, 'True', 0.04570223241397505, 0.03524006622516591, 0.12090050322974853]                             
,                                                                                                                      
[5.252945572624554, 'False', 0.04151499478964532, 0.03230298013245067, 0.1111252775107193]                             
,                                                                                                                      
[43.747472091550165, 'True', 0.045920349437573386, 0.03551655629139097, 0.12103202956643569]                           
,                                                                                                                      
[26.70250127817762, 'True', 0.04614571586579576, 0.03565231788079499, 0.12312758604773234]                             
,                                                                                                                      
[35.46537730183492, 'False', 0.046082001077690185, 0.035584437086093025, 0.12062074545354382]                          
,                                                                                                                      
[50.44222020083109, 'True', 0.04623032073731656, 0.03561754966887439, 0.12275797880287033]                             
,                                                                                                                      
[57.53729528701024, 'False', 0.04537292142111858, 0.03499668874172207, 0.12039952485050845]                            
,                                                                                                                      
[67.46657832871463, 'True', 0.04537972562125557, 0.0350711920529805, 0.12030338783171156]                              
,                                                                                                                      
[31.40370838868003, 'True', 0.04675056437927483, 0.03597682119205334, 0.12431151913486287]                             
,                                                                                                                      
[21.154629029385585, 'True', 0.04638611686778393, 0.03588741721854323, 0.12246096610546696]                            
,                                                                                                                      
[32.13505224452548, 'False', 0.04648455385407856, 0.03598013245033147, 0.12260656351183144]                            
,                                                                                                                      
[7.618634390709651, 'True', 0.04273894551473873, 0.03319867549668897, 0.11313060250210846]
,
[82.95657006612396, 'False', 0.04536342608086675, 0.0348956953642386, 0.121408771116375]                               
,                                                                                                                      
[93.50468566837422, 'False', 0.04442482751307143, 0.034256622516556604, 0.11817531930440403]                           
,                                                                                                                      
[56.02600888724633, 'False', 0.04620414374787472, 0.03566059602649038, 0.12236855478111894]                            
,                                                                                                                      
[5.323964550315936, 'True', 0.04126758950343522, 0.032041390728477157, 0.11028920604874207]                            
,                                                                                                                      
[94.21805967305588, 'True', 0.04501715761543706, 0.03463741721854336, 0.12174023285216977]                             
,                                                                                                                      
[62.98226522012934, 'True', 0.04519662063831233, 0.03485596026490086, 0.12057980588641422]                             
,                                                                                                                      
[16.75390116781267, 'True', 0.04575228925034808, 0.03541887417218574, 0.12063414415725698]                             
,                                                                                                                      
[58.30097044334039, 'False', 0.045899136775297514, 0.035408940397351235, 0.12249093037778618]                          
,                                                                                                                      
[93.9747996499518, 'True', 0.044218975905032785, 0.034129139072848015, 0.11742239549349245]                            
,                                                                                                                      
[8.56159181895043, 'True', 0.04379997440221843, 0.03411258278145732, 0.11568689826156173]                              
,                                                                                                                      
[3.89467587397367, 'False', 0.03795421653439454, 0.02962582781456992, 0.10070315429967991]                             
,                                                                                                                      
[39.21212021925621, 'True', 0.04671228986556218, 0.035991721854304944, 0.12481245537812999]                            
,                                                                                                                      
[67.76170552007106, 'False', 0.04551860711536422, 0.03513576158940423, 0.121604954126342]                              
,                                                                                                                      
[90.8658256183306, 'True', 0.04467325249809896, 0.03448841059602681, 0.11893027245534703]                              
,                                                                                                                      
[21.949322323483397, 'True', 0.04634465624898026, 0.035748344370861156, 0.12309778794419791]                           
,                                                                                                                      
[51.26061964584465, 'False', 0.04554073891861198, 0.03519205298013276, 0.12040425629024663]                            
,                                                                                                                      
[93.53369892254867, 'True', 0.044937620747240226, 0.034538079470198886, 0.12065524988819182]                           
,                                                                                                                      
[81.08781905299796, 'True', 0.045071868160976525, 0.0346456953642388, 0.11989263755214984]                             
,                                                                                                                      
[12.593154160809437, 'True', 0.04510404866339971, 0.0348774834437089, 0.11949926635516116]                             
,                                                                                                                      
[53.16490101929369, 'False', 0.046483595591820345, 0.03587582781456974, 0.1224736023438894]                            
,                                                                                                                      
[42.58488271938388, 'False', 0.046199162341787614, 0.03565397350993414, 0.1209150111972598]                            
,                                                                                                                      
[36.144878508129786, 'False', 0.04653221317940361, 0.03597847682119227, 0.12263521553775517]                           
,                                                                                                                      
[32.37498990376022, 'False', 0.04660632980561137, 0.03596854304635788, 0.12353583352345206]                            
,                                                                                                                      
[35.07509476574636, 'False', 0.04649376067903942, 0.03589735099337779, 0.12208835450115693]                            
,                                                                                                                      
[26.38410961179661, 'True', 0.046879411770766446, 0.03617549668874192, 0.12453290414901716]                            
,                                                                                                                      
[24.853419856060036, 'True', 0.04695872196707718, 0.036327814569536654, 0.12399462107394656]                           
,                                                                                                                      
[26.005218391530647, 'True', 0.046606630297524224, 0.03602980132450355, 0.12347078916117075]                           
,                                                                                                                      
[27.275851134649464, 'True', 0.04643908832976438, 0.03589238410596046, 0.1216150075811862]                             
,                                                                                                                      
[17.703483706712923, 'True', 0.04644894240660368, 0.03595364238410613, 0.12303239358634714]                            
,                                                                                                                      
[45.99805566615065, 'True', 0.04657483134239633, 0.03598344370860954, 0.12234760010775828]                             
,                                                                                                                      
[30.854327550870877, 'True', 0.04661657701429741, 0.03598675496688772, 0.12347919691687975]                            
,                                                                                                                      
[70.77648235003822, 'True', 0.045406819078745005, 0.03499503311258301, 0.12177283991497467]                            
,                                                                                                                      
[22.376429783917246, 'True', 0.04676828862887344, 0.036162251655629404, 0.1239872256485483]                            
,                                                                                                                      
[1.3877745692450816, 'True', 0.030398297121670602, 0.023768211920530236, 0.08011139904830385]                          
,                                                                                                                      
[45.143353300485714, 'True', 0.045819978051663844, 0.03537913907284796, 0.12098523238984293]                           
,                                                                                                                      
[10.21170430542612, 'True', 0.04483549639346289, 0.03475827814569559, 0.1191649311445208]                              
,                                                                                                                      
[13.822730889326921, 'True', 0.04552056849923391, 0.03511589403973532, 0.12219548864056698]                            
,                                                                                                                      
[20.0644955834894, 'True', 0.046090430868346556, 0.03558278145695399, 0.12208875576332733]                             
,                                                                                                                      
[59.03915536713354, 'True', 0.0456245068635171, 0.03520695364238439, 0.12095267867039854]                              
,                                                                                                                      
[25.360385709691833, 'True', 0.04653170224139537, 0.035947019867550035, 0.12371493483661063]                           
,                                                                                                                      
[4.861594307234839, 'True', 0.03990817713207324, 0.031081125827814925, 0.10449546472447828]                            
,                                                                                                                      
[42.041186154212696, 'True', 0.046186212341216136, 0.03557284768211948, 0.12280277280805557]                           
,                                                                                                                      
[30.137821335855918, 'True', 0.04640892982946754, 0.035733443708609494, 0.12226365220790225]]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','F1','Precision','Recall']
metric.head(5)

,neighbours,center,F1,Precision,Recall
0,82.918991,False,0.044490,0.034194,0.118936
1,52.202021,False,0.045544,0.035164,0.120646
2,7.228034,True,0.042851,0.033237,0.114722
3,47.790239,True,0.045962,0.035445,0.121920
4,4.953022,False,0.039646,0.030879,0.104809


In [6]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,neighbours,center,F1,Precision,Recall
21,25.973883,True,0.047110,0.036447,0.123873
82,24.853420,True,0.046959,0.036328,0.123995
81,26.384110,True,0.046879,0.036175,0.124533
89,22.376430,True,0.046768,0.036162,0.123987
53,31.403708,True,0.046751,0.035977,0.124312
20,29.188149,True,0.046716,0.036088,0.123564
68,39.212120,True,0.046712,0.035992,0.124812
22,21.976098,True,0.046700,0.036008,0.124287
39,17.100419,True,0.046620,0.036022,0.124396
87,30.854328,True,0.046617,0.035987,0.123479


In [7]:
metric.to_csv('metric_F1.csv', index=False)